In [3]:
import json

dataset_name = "./outputs/vllm_test_par_batch_out_date_00_00_April_02_2025_Qwen2.5-3B-Instruct_bfloat16_0.json"

#Open and read the JSON file
with open(dataset_name, 'r') as file: 
    data_1 = json.load(file)

In [4]:
template_person = """
{
        "Name": "",
		"Birth_Date": "",
		"Age": "",
		"Height": "",
		"Weight": "",
		"Gender": "",
		"Marital_Status": "",
		"Number_of_Children": "",
		"Place_of_Birth": "",
		"Mother_Maiden_Name": "",
		"Sexual_Preference": "",
		"Sex_Life": ""
}
"""

template_location = """
{
		"Home_Town_City": "",
		"Geographical_Indicators": "",
		"Geo_Location": "",
		"Country": "",
		"ZIP_Code": "",
		"Address": "",
		"Date_Time": ""
}

"""

template_contact = """
{
		"Home_Address": {
			"Street_Address": "",
			"City": "",
			"State": "",
			"ZIP_Code": "",
			"Country": ""
		},
		"Phone_Number": "",
		"Email_Address": "",
		"Family_Friend_Contact_Information": ""
}
"""

template_identifiers = """
{
    "Personal_Identifiers":{
        "National_ID": "",
		"Passport_Number": "",
		"Social_Security_Number": "",
        "Vehicle_Registration": ""
    },  
	"Online_Identifiers": {
		"Screen_Name": "",
		"Social_Network_Profile": "",
		"Social_Network_Activity": "",
		"URLs": "",
		"Online_Aliases": "",
        "IPs": ""
	}
}
"""

template_nrp = """
{
        "Nationality_Citizenship": "",
        "Race_Ethnic": "",
    	"Religion": "",
    	"Philosophical_Belief": "",
    	"Political_Affiliation": "",
    	"Trade_Union_Affiliation": ""

}
"""

template_finance = """
{
		"Banking_Details": {
			"Credit_Card_Number": "",
			"Credit_Score": "",
			"ABA_Routing_Number": "",
			"Bank_Account_Number": "",
			"Individual_Taxpayer_Identification": "",
			"SWIFT_Code": "",
			"Crypto": ""
		},
		"Invoice_Payments": "",
		"Financial_Information": ""
}
"""

template_security = """
{
		"Digital_Signature": "",
		"Password": "",
		"License_Numbers": {
			"Drivers_License_Number": "",
			"Vehicle_Registration_Number": "",
			"License_Plate_Number": ""
		},
		"Biometric_Data": {
			"Fingerprint_Data": "",
			"Voice_Print": "",
			"Handwriting_Sample": "",
			"Physiological_Data": "",
			"Genetic_Data": "",
			"X_Ray": ""
		}
}
"""

template_others = """
{
    "Health_Information": {
		"Health_Insurance_ID": "",
		"Medical_History": "",
		"Physiological_Data": ""
	},
	"Cultural_and_Social_Identity": {
		"Cultural_Social_Identity": "",
		"Shopping_Behavior": "",
		"Survey_Answers": "",
		"Signed_Petitions": "",
		"Activities": "",
		"Law_Enforcement_Records": ""
	},
    "Appearance": {
		"Picture_of_Face": "",
		"Distinguishing_Characteristic": ""
	}
}
"""

template_work = """
{
		"Job_Title": "",
		"Occupation": "",
		"Work_ID": "",
		"Work_Address": {
			"Street_Address": "",
			"City": "",
			"State": "",
			"ZIP_Code": "",
			"Country": ""
		},
		"Work_Contact_Information": {
			"Work_Phone_Number": "",
			"Work_Email_Address": ""
		},
		"Employment_Information": {
			"Employment_Status": "",
			"Work_Experience": "",
			"Skills": "",
			"Education": ""
    	},
    	"Income_Level": ""
}
"""

In [9]:
from json_repair import repair_json
def repair_output(in_a):
    if  len(in_a) < 10:
        return ""
    if in_a.count('}') ==  in_a.count('{'):
        return in_a
    elif in_a.count('}') > in_a.count('{'):
        try:
            json.loads("{" + in_a)
            in_a = "{" + in_a
        except:
            try:
                json.loads(in_a[1:])
                in_a = in_a[1:]
            except:
                try:
                    in_a = repair_json(in_a)
                    print("Repaired Output")
                    return in_a
                except:
                    #print("Output not recoverable {")
                    return ""
                
        return in_a
    elif in_a.count('}') < in_a.count('{'):
        try:
            json.loads( in_a + "}")
            in_a =  in_a + "}"
        except:
            try:
                json.loads(in_a[:-1])
                in_a = in_a[:-1]
            except:
                #print("Output not recoverable }")
                return ""
                
def define_structure(in_a,template):
    if isinstance(in_a,dict) and 'Criminal_Record' in in_a:
        del in_a['Criminal_Record']
    if in_a.keys() != template.keys():
        #print(f"{in_a.keys()}")
        #print(f"{template.keys()}" )
        return template
    out = {} 
    for key in template:
        if isinstance(template[key],dict) and isinstance(in_a[key],dict):
            out[key] = define_structure(in_a[key], template[key])
        elif not isinstance(in_a[key], dict) and not isinstance(template[key], dict):
            out[key]  = in_a[key]
        else:
            out[key] = template[key]
    return out
def handle_broken_output(in_a, template):
    in_a = repair_output(in_a)
    try:
        out = define_structure(dict(json.loads(in_a)),dict(json.loads(template)))
    except:
        return template
    return out

def sum_dicts(a,b):
    for i,key in enumerate(b):
        if isinstance(b[key], int):
            a[key] = a[key]+b[key]
        if isinstance(b[key], dict):
            a[key] = sum_dicts(a[key],b[key])
    return a
    
def check_output(dic):
    w = {}
    if not isinstance(dic,dict):
        try:
            dic = dict(json.loads(dic))
        except:
            print(f"Could not load {dic}")
    for key in dic:
        if (key == 'Criminal_Record'):
            continue
        if isinstance(dic[key], dict):
            w[key]=check_output(dic[key])
        else:
            if dic[key] != "":
                w[key]= 1
            else:
                w[key] = 0
    return w    

In [10]:

def get_data(data,template, ID ='', depth = 0):
    if isinstance(data,dict):
        get_data_sub(data,template,ID,depth)
    elif isinstance(data, list):
        get_data_sub_2(data,template,ID,depth)
        
                     
    return

def get_data_sub(data,template, ID ='', depth = 0):
    print("Brrrr")
    for i,dat in enumerate(data):
        ID = dat
        #print(dat)
        for da in data[dat]:
            templates_name = ['PERSON','LOCATION','CONTACT','IDENTIFIERS','NRP','FINANCE','SECURITY','WORK','OTHERS']
            data_name = ["Person","Location","Contact","Identifiers","NRP","Finance","Security","Work","Others"]
            for iii,n in enumerate(templates_name):
                if ID not in out_conv.keys():
                    out_conv[ID] = {}
                    out = handle_broken_output(da['out'][data_name[iii]],templates[iii])
                    out_conv[ID][n] = check_output(out)
                else:
                    if n not in out_conv[ID].keys():
                        out = handle_broken_output(da['out'][data_name[iii]],templates[iii])
                        out_conv[ID][n] = check_output(out)
                    else:    
                        out = check_output(handle_broken_output(da['out'][data_name[iii]],templates[iii]))
                        out_conv[ID][n] = sum_dicts(out_conv[ID][n],out) 
                    
    return

def get_data_sub_2(data,template, ID ='', depth = 0):
    templates_name = ['PERSON','LOCATION','CONTACT','IDENTIFIERS','NRP','FINANCE','SECURITY','WORK','OTHERS']
    for i,dat in enumerate(data):
            ID = dat['ID']
            for ii,da in enumerate(dat['result']):
                    if isinstance(da['data'],str):
                        if ID not in out_conv.keys():
                            out_conv[ID] = {}
                            out = handle_broken_output(d,templates[iii])
                            out_conv[ID][templates_name[iii]] = check_output(out)
                        else:
                            if n not in out_conv[ID].keys():
                                out = handle_broken_output(d,templates[iii])
                                out_conv[ID][templates_name[iii]] = check_output(out)
                            else:    
                                out = check_output(handle_broken_output(d,templates[iii]))
                                out_conv[ID][templates_name[iii]] = sum_dicts(out_conv[ID][templates_name[iii]],out) 
                    
                    else:
                        if data[i]['result'][ii]['data'] == []: continue
                        for iii,n in enumerate(templates_name):
                            #print(f"{i} {ii} {iii}  {ID}")
                            #print(iii)
                            if ID not in out_conv.keys():
                                out_conv[ID] = {}
                                out = handle_broken_output(data[i]['result'][ii]['data'][iii]['data'],templates[iii])
                                out_conv[ID][n] = check_output(out)
                            else:
                                if n not in out_conv[ID].keys():
                                    out = handle_broken_output(data[i]['result'][ii]['data'][iii]['data'],templates[iii])
                                    out_conv[ID][n] = check_output(out)
                                else:    
                                    out = check_output(handle_broken_output(data[i]['result'][ii]['data'][iii]['data'],templates[iii]))
                                    out_conv[ID][n] = sum_dicts(out_conv[ID][n],out) 
                    
    return


In [11]:
import time
out_conv = {}
templates = [template_person,
            template_location,
            template_contact,
            template_identifiers,
            template_nrp,
            template_finance,
            template_security,
            template_work,
            template_others]

get_data(data_1,templates)


Brrrr


In [17]:
output_file = "./outputs/Processed/Qwen25_3B_Presidio_bfloat_cap_v1.json"

with open(output_file, 'w') as f:
    json.dump(out_conv, f, indent=4)


data_1[data_1['id'] == "E0YL5SX"]

In [125]:
!pip install json_repair

In [ ]:


 if data_1[kk]['PERSON']['Name'] > 0: #PERSON
        d['PERSON'] += 1
    if data_1[kk]['CONTACT']['Email_Address'] > 0 or data_1[kk]['WORK']['Work_Contact_Information']['Work_Email_Address'] > 0 : #EMAIL_ADDRESS
        d['EMAIL_ADDRESS'] += 1
    if data_1[kk]['CONTACT']['Home_Address'] != [0,0,0,0,0] or data_1[kk]['LOCATION'] != [0,0,0,0,0,0,0] : #LOCATION
        d['LOCATION'] += 1
    if data_1[kk]['NRP']['Nationality_Citizenship'] > 0 or data_1[kk]['NRP']['Race_Ethnic'] > 0 or data_1[kk]['NRP']['Religion'] > 0 or data_1[kk]['NRP']['Philosophical_Belief'] > 0 or data_1[kk]['NRP']['Political_Affiliation'] > 0:# NRP
        d['NRP'] += 1
    if data_1[kk]['PERSON']['Birth_Date'] > 0 or data_1[kk]['LOCATION']['Date_Time'] > 0: #DATE_TIME
        d['DATE_TIME'] += 1
    if data_1[kk]['IDENTIFIERS']['Personal_Identifiers']['Passport_Number'] > 0 or data_1[kk]['IDENTIFIERS']['Personal_Identifiers']['National_ID'] > 0 :
        d['PASSPORT_NUMBER'] += 1 #PASSPORT_NUMBER
    if data_1[kk]['CONTACT']['Phone_Number'] > 0 or data_1[kk]['WORK']['Work_Contact_Information']['Work_Phone_Number'] > 0: #PHONE_NUMBER
        d['PHONE_NUMBER'] += 1
    if data_1[kk]['IDENTIFIERS']['Online_Identifiers']['URLs'] > 0: #URL
        d['URL'] +=1
    dd[kk] = d



table_eqi['PERSON'] = {'PERSON':['Name',
                                 'Mother_Maiden_Name'],
                       'CONTACT':['Family_Friend_Contact_Information']
                      }


table_eqi['LOCATION'] = {'PERSON':['Place_of_Birth'],
                         'LOCATION': ['Home_Town_City',
                                      'Country',
                                      'Address'],
                         'CONTACT':{'Home_Address': ['Street_Address',
                                                     'City',
                                                     'State',
                                                     'Country']
                                   }
                        }

table_eqi['EMAIL'] = {'CONTACT':['Email_Address'],
                          'WORK': {'Work_Contact_Information':['Work_Email_Address']
                                  }
                     }

table_eqi['DATE_TIME'] = {'PERSON':['Birth_Date'],
                          'LOCATION':['Date_Time']
                         }

table_eqi['NRP'] = {'NRP':['Nationality_Citizenship',
                           'Race_Ethnic',
                           'Religion',
                           'Philosophical_Belief',
                           'Political_Affiliation',
                           'Trade_Union_Affiliation']
                   }

table_eqi['PASSPORT'] = {'IDENTIFIERS':{'Personal_Identifiers': ['Passport_Number']
                                       },
                        }

table_eqi['PHONE_NUMBER'] = {'CONTACT':['Phone_Number'],
                             'WORK':{'Work_Contact_Information':['Work_Phone_Number']
                                    }
                            }

table_eqi['URL'] = {'IDENTIFIERS':{'Online_Identifiers': ['URLs']
                                       },
                        }











